# Flash Point: Fire Rescue

## Instalación e importación de librerías

In [1]:
# Descargar e instalar mesa, seaborn y plotly

%pip install mesa==2.3.1 seaborn plotly --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model

# Debido a que necesitamos que existan múltiples agentes, importamos la clase 'MultiGrid'.
from mesa.space import MultiGrid

# Con 'SimultaneousActivation' podemos activar todos los agentes al mismo tiempo.
from mesa.time import SimultaneousActivation

# Con 'RandomActivation' podemos activar los agentes en un orden aleatorio.
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# BATCH_RUNNER
from mesa.batchrunner import batch_run

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

# Importamos el paquete seaborn para mejorar la visualización de los datos.
import seaborn as sns

# Importamos el paquete plotly para crear gráficos interactivos.
import plotly.graph_objects as go

# Importamos el paquete networkx para crear grafos.
import networkx as nx

## Leer archivo de configuración inicial

In [3]:
# Abrir el archivo de configuración
file = open("board-config.txt", "r")

# Obtener la información del tablero
config = file.readlines()

# Cerrar el archivo
file.close()

# Crear un diccionario para almacenar la configuración del tablero
board_config = {}

# Configuración del tablero
board_config['board'] = [x.replace("\n", "").split() for x in config[:6]]

# Puntos de interés
board_config['points_of_interest'] = [x.replace("\n", "").split() for x in config[6:9]]

# Indicadores de fuego
board_config['fire_indicators'] = [x.replace("\n", "").split() for x in config[9:19]]

# Puertas
board_config['doors'] = [x.replace("\n", "").split() for x in config[19:27]]

# Puntos de entrada
board_config['entry_points'] = [x.replace("\n", "").split() for x in config[27:31]]

# Imprimir la configuración del tablero
board_config

{'board': [['1100', '1000', '1001', '1100', '1001', '1100', '1000', '1001'],
  ['0100', '0000', '0011', '0100', '0011', '0110', '0010', '0011'],
  ['0100', '0001', '1100', '1000', '1000', '1001', '1100', '1001'],
  ['0100', '0011', '0110', '0010', '0010', '0011', '0110', '0011'],
  ['1100', '1000', '1000', '1000', '1001', '1100', '1001', '1101'],
  ['0110', '0010', '0010', '0010', '0011', '0110', '0011', '0111']],
 'points_of_interest': [['2', '4', 'v'], ['5', '1', 'f'], ['5', '8', 'v']],
 'fire_indicators': [['2', '2'],
  ['2', '3'],
  ['3', '2'],
  ['3', '3'],
  ['3', '4'],
  ['3', '5'],
  ['4', '4'],
  ['5', '6'],
  ['5', '7'],
  ['6', '6']],
 'doors': [['1', '3', '1', '4'],
  ['2', '5', '2', '6'],
  ['2', '8', '3', '8'],
  ['3', '2', '3', '3'],
  ['4', '4', '5', '4'],
  ['4', '6', '4', '7'],
  ['6', '5', '6', '6'],
  ['6', '7', '6', '8']],
 'entry_points': [['1', '6'], ['3', '1'], ['4', '8'], ['6', '3']]}

## Solución aleatoria

In [4]:
def add_fire(G, x, y):
    """
    Agregar fuego a una celda
    """

    # Verificar si la celda existe
    if (x, y) not in G.nodes:
        return 
    
    # Colocar fuego
    G.nodes[(x, y)]['fire'] = 2

    # Cambia el peso de las aristas adyacentes a la celda a 3
    # Significa que al agente le costará el doble de tiempo pasar por esa arista
    # Moverse en una celda con fuego cuesta 2 puntos de acción
    for neighbor in G.adj[(x, y)]:
        if G.get_edge_data((x, y), neighbor)['type'] == 'path':
            G[(x, y)][neighbor]['weight'] = 3


def initialize_board(board_config):
    """
    De acuerdo a la configuración del tablero, se inicializa el tablero como un grafo
    """

    # Crear un grafo vacío
    G = nx.Graph()

    # Información del tablero (transiciones y muros)
    # "TLDR" -> Top, Left, Down, Right
    board_info = board_config['board']

    # Puertas
    doors = {}
    for door in board_config['doors']:
        # Se resta 1 a las coordenadas para que coincidan con las coordenadas del tablero
        doors[(int(door[0]) - 1, int(door[1]) - 1)] = (int(door[2]) - 1, int(door[3]) - 1)

    # Agregar los nodos al grafo
    for i in range(len(board_info)):
        for j in range(len(board_info[i])):
            G.add_node((i, j), fire=0, POI=None, isEntryPoint=False)

    # Agregar las aristas al grafo
    for i in range(len(board_info)):
        for j in range(len(board_info[i])):

            # String con la información de la celda
            # "TLDR" -> Top, Left, Down, Right
            # Peso 1 -> Libre / Puerta abierta o destruida
            # Peso 2 -> Puerta cerrada
            # Peso 5 -> Pared
            
            # Verificar si hay una transición arriba
            if board_info[i][j][0] == '0':
                G.add_edge((i, j), (i - 1, j), weight = 1, type='path')
            else:
                # Verificar si hay una puerta
                if (i, j) in doors and doors[(i, j)] == (i - 1, j):
                    # Agregar puerta
                    G.add_edge((i, j), (i - 1, j), weight = 2, type='door')
                # Si no hay puerta, agregar pared
                elif i != 0 and not G.has_edge((i, j), (i - 1, j)):
                    G.add_edge((i, j), (i - 1, j), weight = 5, type='wall', life=2)


            # Verificar si hay una transición a la izquierda
            if board_info[i][j][1] == '0':
                G.add_edge((i, j), (i, j - 1), weight = 1, type='path')
            else:
                # Verificar si hay una puerta
                if (i, j) in doors and doors[(i, j)] == (i, j - 1):
                    # Agregar puerta
                    G.add_edge((i, j), (i, j - 1), weight = 2, type='door')
                # Si no hay puerta, agregar pared
                elif j != 0 and not G.has_edge((i, j), (i, j - 1)):
                    G.add_edge((i, j), (i, j - 1), weight = 5, type='wall', life=2)

            # Verificar si hay una transición abajo
            if board_info[i][j][2] == '0':
                G.add_edge((i, j), (i + 1, j), weight = 1, type='path')
            else:
                # Verificar si hay una puerta
                if (i, j) in doors and doors[(i, j)] == (i + 1, j):
                    # Agregar puerta
                    G.add_edge((i, j), (i + 1, j), weight = 2, type='door')
                # Si no hay puerta, agregar pared
                elif i != len(board_info) - 1 and not G.has_edge((i, j), (i + 1, j)):
                    G.add_edge((i, j), (i + 1, j), weight = 5, type='wall', life=2)

            # Verificar si hay una transición a la derecha
            if board_info[i][j][3] == '0':
                G.add_edge((i, j), (i, j + 1), weight = 1, type='path')
            else:
                # Verificar si hay una puerta
                if (i, j) in doors and doors[(i, j)] == (i, j + 1):
                    # Agregar puerta
                    G.add_edge((i, j), (i, j + 1), weight = 2, type='door')
                # Si no hay puerta, agregar pared
                elif j != len(board_info[i]) - 1 and not G.has_edge((i, j), (i, j + 1)):
                    G.add_edge((i, j), (i, j + 1), weight = 5, type='wall', life=2)

    # Agregar los puntos de interés
    # None -> No es un punto de interés
    # True -> Hay una víctima
    # False -> Es una falsa alarma
    for poi in board_config['points_of_interest']:
        G.nodes[(int(poi[0]) - 1, int(poi[1]) - 1)]['POI'] = True if poi[2] == 'v' else False

    # Agregar los indicadores de fuego
    # 0 -> No hay fuego
    # 1 -> Hay humo
    # 2 -> Hay fuego
    for fire in board_config['fire_indicators']:
        add_fire(G, int(fire[0]) - 1, int(fire[1]) - 1)

    # Agregar los puntos de entrada
    for entry_point in board_config['entry_points']:
        G.nodes[(int(entry_point[0]) - 1, int(entry_point[1]) - 1)]['isEntryPoint'] = True


    # Retornar el grafo
    return G

def plot_graph(G):
    """
    Graficar el grafo con Plotly
    """

    # Definir las posiciones de los nodos como una cuadrícula
    pos = {node: (node[1], -node[0]) for node in G.nodes()}  # El eje y se invierte para que la visualización sea de arriba a abajo

    # Crear trazas para las aristas con colores según su tipo y mostrar peso y tipo
    edge_x = []
    edge_y = []
    edge_colors = []
    edge_annotations = []

    # Definir los colores para cada tipo de arista
    edge_type_colors = {
        'wall': 'red',
        'path': 'green',
        'door': 'blue',
        'unknown': 'gray'  # Color por defecto para tipos no definidos
    }

    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)  # Para separar las líneas de las aristas
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

        # Asignar color según el tipo de la arista
        edge_type = edge[2].get('type', 'unknown')  # 'unknown' si no tiene tipo
        edge_colors.append(edge_type_colors.get(edge_type, 'gray'))  # Usar color definido o 'gray'

        # Guardar anotaciones para peso y tipo
        weight = edge[2].get('weight', '?')
        edge_annotations.append(
            dict(
                x=(x0 + x1) / 2,
                y=(y0 + y1) / 2,
                text=f'{weight}<br>{edge_type}',  # Mostrar peso y tipo
                showarrow=False,
                font=dict(size=10, color='black')
            )
        )

    # Crear trazas para las aristas coloreadas
    edge_traces = []
    for idx in range(len(edge_colors)):
        edge_traces.append(
            go.Scatter(
                x=[edge_x[idx * 3], edge_x[idx * 3 + 1], None],
                y=[edge_y[idx * 3], edge_y[idx * 3 + 1], None],
                line=dict(width=2, color=edge_colors[idx]),
                hoverinfo='none',
                mode='lines'
            )
        )

    # Crear trazas para los nodos y asignar colores según las condiciones
    node_x = []
    node_y = []
    node_colors = []
    node_text = []

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        # Determinar el color según las condiciones
        if G.nodes[node].get("fire") == 1:  # Humo
            node_colors.append('orange')
        elif G.nodes[node].get("fire") == 2:  # Fuego
            node_colors.append('red')
        elif G.nodes[node].get("POI") is not None:  # POI
            node_colors.append('blue')
        elif G.nodes[node].get("isEntryPoint"):  # Punto de entrada
            node_colors.append('green')
        else:
            node_colors.append('black')  # Color por defecto

        # Crear texto para hover
        adyacentes = []
        for neighbor in G.adj[node]:
            edge_data = G.get_edge_data(node, neighbor)
            edge_type = edge_data.get('type', 'unknown')
            adyacentes.append(f'{neighbor}: {edge_type}')

        node_text.append(
            f'Posición: {node}<br>'
            f'Adyacentes: {", ".join(adyacentes)}<br>'
            f'Fuego: {G.nodes[node].get("fire")}<br>'
            f'POI: {G.nodes[node].get("POI")}<br>'
            f'Es punto de entrada: {G.nodes[node].get("isEntryPoint", False)}'   
        )

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            size=10,
            color=node_colors,  # Colores asignados según las condiciones
        ),
        text=node_text
    )

    # Crear la figura
    fig = go.Figure(
        data=edge_traces + [node_trace],  # Agregar trazas de aristas coloreadas y nodos
        layout=go.Layout(
            title='Flash Point: Fire Rescue',
            titlefont_size=16,
            showlegend=False,
            hovermode='closest',
            margin=dict(b=20, l=5, r=5, t=40),
            annotations=edge_annotations,  # Agregar anotaciones para pesos y tipos
            xaxis=dict(showgrid=False, zeroline=False),
            yaxis=dict(showgrid=False, zeroline=False)
        )
    )

    fig.show()

def place_smoke(G, x, y):
    """
    Colocar humo en una celda y resuelve la propagación del fuego (igniciones y explosiones)
    """

    # Verificar si la celda existe
    if (x, y) not in G.nodes:
        return 
    
    # Verificar si no hay fuego
    if G.nodes[(x, y)]['fire'] == 0:
        G.nodes[(x, y)]['fire'] = 1

    # Verificar si hay humo (Ignición)
    # Todo el humo en contacto con fuego se convierte en fuego
    elif G.nodes[(x, y)]['fire'] == 1:

        # Convertir el humo en fuego
        add_fire(G, x, y)

        # Obtener los vecinos de la celda
        neighbors = G.adj[(x, y)]

        # Verificar si hay humo en los vecinos
        for neighbor in neighbors:

            # ¿Hay humo y no hay una pared entre las celdas?
            if G.nodes[neighbor]['fire'] == 1 and G.get_edge_data((x, y), neighbor)['type'] != 'wall':

                # Convertir el humo en fuego recursivamente
                place_smoke(G, neighbor[0], neighbor[1])

    # Verificar si hay fuego (Explosión)
    # Expande el fuego en todas las direcciones (Von Neumann) si no hay una pared
    ...    

In [5]:
# Inicializar el tablero
G = initialize_board(board_config)

print("Inicial")
plot_graph(G)

place_smoke(G, 2, 0)
place_smoke(G, 3, 0)
place_smoke(G, 3, 1)
place_smoke(G, 4, 1)

print("Con humo")
plot_graph(G)

place_smoke(G, 3, 1)

print("Simular ignición (Poner humo en (3, 1))")
plot_graph(G)

Inicial


Con humo


Simular ignición (Poner humo en (3, 1))


In [6]:
class BoardModel(Model):

    def __init__(self, board_config):

        # Definir el tamaño del tablero
        self.height = len(board_config['board'])
        self.width = len(board_config['board'][0])

        # Crear el grid
        self.grid = MultiGrid(self.width, self.height, torus=False)

        # Crear grafo para el tablero
        self.graph = nx.Graph()     